In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\Administrator\LIS\akraman_del_model_input.csv', encoding='cp1252')

In [4]:
df1=df[['Aakraman Store Data KYCID','latitude','longitude']]
df1.head()

Aakraman Store Data KYCID   latitude  longitude
0                       5704  28.737042  77.006696
1                       5659  28.562438  77.200457
2                       5646  28.723452  77.247645
3                       5603  28.594908  77.065229
4                       5574  28.678018  77.224954

In [13]:
# Prepare lat-long data and convert to radians
coords = df1[['latitude', 'longitude']].to_numpy()
coords_rad = np.radians(coords)

In [14]:
kms_per_radian = 6371.0088
epsilon = 0.5 / kms_per_radian  # ~0.5 km

In [15]:
db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', metric='haversine')
db.fit(coords_rad)

DBSCAN(algorithm='ball_tree', eps=7.848050688613081e-05, metric='haversine')

In [16]:
df['cluster'] = db.labels_


In [17]:
df['cluster'].value_counts().sort_index()

cluster
-1     612
 0      25
 1       9
 2     249
 3     523
      ... 
 92     15
 93      6
 94      4
 95      5
 96      5
Name: count, Length: 98, dtype: int64

In [19]:
# df.to_csv(r'C:\Users\Administrator\LIS\akraman_del_cluster_outputv5.csv', index=False)

In [23]:
from geopy.distance import geodesic

In [22]:
# pip install geopy

In [24]:
# Function to compute centroid (mean lat, lon) of a cluster
def compute_centroid(cluster_df):
    return (cluster_df['latitude'].mean(), cluster_df['longitude'].mean())

In [25]:
centroids = {}

In [26]:
for cluster_id in df['cluster'].unique():
    if cluster_id == -1:
        continue
    cluster_data = df[df['cluster'] == cluster_id]
    centroids[cluster_id] = compute_centroid(cluster_data)

In [27]:
df['centroid_lat'] = df['cluster'].apply(lambda x: centroids.get(x, (None, None))[0])
df['centroid_lon'] = df['cluster'].apply(lambda x: centroids.get(x, (None, None))[1])

In [28]:
def distance_from_centroid(row):
    if row['cluster'] == -1:
        return None
    return geodesic((row['latitude'], row['longitude']), (row['centroid_lat'], row['centroid_lon'])).km

df['distance_from_centroid_km'] = df.apply(distance_from_centroid, axis=1)

In [30]:
df.to_csv(r'C:\Users\Administrator\LIS\akraman_del_cluster_outputv5.csv', index=False)